In [1]:
from pynq import Overlay
bs = Overlay("/usr/local/lib/python3.6/dist-packages/pynq_cv/overlays/xv2Filter2DDilate.bit")
bs.download()

/usr/local/lib/python3.6/dist-packages/setuptools/distutils_patch.py:26: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "


In [2]:
from pynq import Xlnk
Xlnk.set_allocator_library('/usr/local/lib/python3.6/dist-packages/pynq_cv/overlays/xv2Filter2DDilate.so')
mem_manager = Xlnk()

import pynq_cv.overlays.xv2Filter2DDilate as xv2
import numpy as np
import cv2
import time

In [4]:
img = cv2.imread('/home/xilinx/jupyter_notebooks/eeve.jpeg')
imgY = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

print("Size of imgY is ",imgY.shape);
height, width, channels = img.shape

Size of imgY is  (497, 701)


In [11]:
kernel = np.array([[1.0,2.0,1.0],[0.0,0.0,0.0],[-1.0,-2.0,-1.0]],np.float32)

numberOfIterations=50

dstSW = np.ones((height,width),np.uint8);

xFimgY  = mem_manager.cma_array((height,width),np.uint8) #allocated physically contiguous numpy array 
xFimgY[:] = imgY[:] # copy source data

xFdst  = mem_manager.cma_array((height,width),np.uint8) #allocated physically contiguous numpy array

In [12]:
startSW=time.time()
for i in range(numberOfIterations):
    cv2.filter2D(imgY,-1,kernel,dst=dstSW,borderType=cv2.BORDER_CONSTANT) #filter2D on ARM
stopSW=time.time()
print("Start SW loop = ", (stopSW - startSW))

Start SW loop =  1.5109682083129883


In [13]:
startPL=time.time()
for i in range(numberOfIterations):
    xv2.filter2D(xFimgY,-1,kernel,dst=xFdst,borderType=cv2.BORDER_CONSTANT) #filter2D offloaded to PL, working on physically continuous numpy arrays
stopPL=time.time()
print("Start HW loop = ", (stopPL - startPL))

Start HW loop =  0.18459439277648926


In [14]:
print("SW frames per second: ", ((numberOfIterations) / (stopSW - startSW)))
print("PL frames per second: ", ((numberOfIterations) / (stopPL - startPL)))

SW frames per second:  33.0913646792248
PL frames per second:  270.8641321545506
